# Tidal flow and vegetation interactions

*(modified from Greg Tucker, August 2020)*

This tutorial explains how the `TidalFlowCalculator` Landlab component can be combined with a range of vegetation modeling approaches to describe salt marsh dynamics. We will start with the simplest vegetation formulation and then add complexity to see how the results change. Depending on the simulation temporal scale, sometimes the differences between vegetation modeling approaches may not affect the results but over time, the integrated ecogeomorphic dynamics become more pronounced and more sophisticated models of vegetation become necessary to capture the role of plants in the salt marsh system.

## Theory

The `TidalFlowCalculator` computes a tidal-cycle averaged flow velocity field, given a topography (bathymetry), mean sea level, tidal range, and tidal period. The approach that the component uses is based on Mariotti (2018). The idea is to calculate a flow velocity field that is just sufficient to bring in (flood tide) or send out (ebb tide) all of the water that enters or leaves the system during one tidal cycle. We will not discuss the numerical background of this component here. For more information, see the Tidal Flow Calculator Landlab tutorial.

In reality, tidal flow across a wetland landscape depends on the underlying bathymtery and roughness that is largely self-organized by ecological processes. Salt marsh environments are a mosaic of marsh plains, tidal channels, and unvegetated pools. The interaction of tides, waves, and storms and the resultant sediment transport which occur at temporal timescales on the order of seconds to hours interact with biogeochemical and ecological processes which occur at timescales on the order of days to years to form and maintain this unique ecotone.

# Creating the domain

First we will start with a simple gridded domain that consists of a flat marsh plain intersected by a single channel, identical to a case presented in the Tidal Flow Calculator tutorial. The first codeblocks import the libraries we will need and create our plotting functions.

In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from landlab import RasterModelGrid, imshow_grid
from landlab.components import TidalFlowCalculator

In [ ]:
from landlab.grid.mappers import map_link_vector_components_to_node


def map_velocity_components_to_nodes(grid):
    """Map the velocity components from the links to the nodes, and return the node arrays."""
    ebb_vel_x, ebb_vel_y = map_link_vector_components_to_node(
        grid, grid.at_link["ebb_tide_flow__velocity"]
    )
    flood_vel_x = -ebb_vel_x
    flood_vel_y = -ebb_vel_y
    return (ebb_vel_x, ebb_vel_y, flood_vel_x, flood_vel_y)


def plot_tidal_flow(grid, resample=1):
    (ebb_x, ebb_y, flood_x, flood_y) = map_velocity_components_to_nodes(grid)

    # depth
    plt.figure()
    imshow_grid(
        grid, grid.at_node["mean_water__depth"], cmap="YlGnBu", color_for_closed="g"
    )
    plt.title("Water depth (m)")
    plt.xlabel("Distance (m)")
    plt.ylabel("Distance (m)")

    # elevation 
    plt.figure()
    imshow_grid(
        grid, grid.at_node["topographic__elevation"], cmap="YlGnBu", color_for_closed="g"
    )
    plt.title("Topo elev (m)")
    plt.xlabel("Distance (m)")
    plt.ylabel("Distance (m)")

    # down-sample for legible quiver plots if needed
    if resample != 1:
        xr = grid.x_of_node.reshape(
            (grid.number_of_node_rows, grid.number_of_node_columns)
        )[::resample, ::resample]
        yr = grid.y_of_node.reshape(
            (grid.number_of_node_rows, grid.number_of_node_columns)
        )[::resample, ::resample]
        ebb_xr = ebb_x.reshape((grid.number_of_node_rows, grid.number_of_node_columns))[
            ::resample, ::resample
        ]
        ebb_yr = ebb_y.reshape((grid.number_of_node_rows, grid.number_of_node_columns))[
            ::resample, ::resample
        ]
        fld_xr = flood_x.reshape(
            (grid.number_of_node_rows, grid.number_of_node_columns)
        )[::resample, ::resample]
        fld_yr = flood_y.reshape(
            (grid.number_of_node_rows, grid.number_of_node_columns)
        )[::resample, ::resample]
    else:
        xr = grid.x_of_node
        yr = grid.y_of_node
        ebb_xr = ebb_x
        ebb_yr = ebb_y
        fld_xr = flood_x
        fld_yr = flood_y

    # ebb tide
    plt.figure()
    imshow_grid(grid, grid.at_node["topographic__elevation"])
    plt.quiver(xr, yr, ebb_xr, ebb_yr)
    plt.title("Ebb Tide")
    plt.xlabel("Distance (m)")
    plt.ylabel("Distance (m)")

    ebb_vel_magnitude = np.sqrt(ebb_x * ebb_x + ebb_y * ebb_y)
    plt.figure()
    imshow_grid(grid, ebb_vel_magnitude, cmap="magma", color_for_closed="g")
    plt.title("Ebb Tide Velocity Magnitude (m/s)")
    plt.xlabel("Distance (m)")
    plt.ylabel("Distance (m)")

    # flood tide
    plt.figure()
    imshow_grid(grid, grid.at_node["topographic__elevation"])
    plt.quiver(xr, yr, fld_xr, fld_yr)
    plt.title("Flood Tide")
    plt.xlabel("Distance (m)")
    plt.ylabel("Distance (m)")

    plt.figure()
    flood_vel_magnitude = np.sqrt(flood_x * flood_x + flood_y * flood_y)
    imshow_grid(grid, flood_vel_magnitude, cmap="magma", color_for_closed="g")
    plt.title("Flood Tide Velocity Magnitude (m/s)")
    plt.xlabel("Distance (m)")
    plt.ylabel("Distance (m)")

In this example, vegetation is assumed to occur only on the marsh plain and is uniformly dense such that the Manning's roughness coefficient is equal to 0.2.

In [ ]:
from landlab.grid.mappers import map_max_of_link_nodes_to_link

# parameters
nrows = 400
ncols = 200
grid_spacing = 2.0  # m
marsh_height = 0.6  # m
channel_depth = 0.6  # m
tidal_range = 1.2192  # m
tidal_period = 12.5 * 3600.0  # s
open_nodes = np.arange(
    94, 105, dtype=int
)  # IDs of open-boundary nodes (along channel at bottom/south boundary)
roughness_shallow = 0.2  # Manning's n for areas above mean sea level (i.e., the marsh)
roughness_deep = 0.01  # Manning's n for areas below mean sea level (i.e., the channel)

# create and set up the grid
grid = RasterModelGrid((nrows, ncols), xy_spacing=grid_spacing)
z = grid.add_zeros("topographic__elevation", at="node", clobber=True)
z[grid.core_nodes] = marsh_height
channel = np.logical_and(grid.x_of_node >= 188.0, grid.x_of_node <= 208.0)
z[channel] = -channel_depth
grid.set_closed_boundaries_at_grid_edges(True, True, True, True)
grid.status_at_node[open_nodes] = grid.BC_NODE_IS_FIXED_VALUE

# set up roughness field (calculate on nodes, then map to links)
roughness_at_nodes = roughness_shallow + np.zeros(z.size)
roughness_at_nodes[z < 0.0] = roughness_deep
roughness = grid.add_zeros("roughness", at="link", clobber=True)
estimated_biomass = grid.add_zeros("biomass", at="node", clobber=True)
map_max_of_link_nodes_to_link(grid, roughness_at_nodes, out=roughness)

# instantiate the TidalFlowCalculator
tfc = TidalFlowCalculator(
    grid, tidal_range=tidal_range, tidal_period=tidal_period, roughness="roughness"
)

# run it
tfc.run_one_step()

# make plots...
plot_tidal_flow(grid, resample=10)

This shows a snapshot of peak velocities at flood and ebb tide and the maximum depth of water at high tide. 

However, marsh plains are not uniformly flat and serve to connect to the uplands. Let's adjust our bathymetry to reflect a mild slope between the channel edge and the boundary.

In [ ]:
marsh_max_height = 0.75
marsh_min_height = 0.25
marsh_slope = (marsh_max_height - marsh_min_height)/(400 - 208)
z[grid.x_of_node < 188] = marsh_max_height - marsh_slope * grid.x_of_node[grid.x_of_node < 188]
z[grid.x_of_node > 208] = marsh_min_height + marsh_slope * (grid.x_of_node[grid.x_of_node > 208] - 208)

# instantiate the TidalFlowCalculator
tfc_slope = TidalFlowCalculator(
    grid, tidal_range=tidal_range, tidal_period=tidal_period, roughness="roughness"
)

# run it
tfc_slope.run_one_step()

# make plots...
plot_tidal_flow(grid, resample=10)

As you can see, adding a sloped marsh plain concentrates zone of measurable tidal velocity closer to the channel. However, the zone of greatest velocity remains in the channel. The maximum depth is increased closer to the channel but the change over one tidal cycle in subtle. In actuality, vegetation productivity on the marsh plain is strongly controlled by the inundation duration. In many simple ecogeomorphic marsh models, the relationship is described with a parabolic deterministic relationship between inundation and peak biomass. The peak biomass directly relates to the roughness.

First we need to calculate the inundation time per tidal period. This equation taken from Hickey et al 2019 and used by the Cohort Marsh Equilibrium Model (Vahesen et al., 2024) is used to calculate the inundation time as a function of tidal datums and elevation.

In [ ]:
#' Calculate flood time using trigonometric method
#'
#' This function calculates flood time in hours from an elevation, and a high and low tidal datum. 
#' @param z a numeric, elevation of the marsh
#' @param datumHigh a numeric, highest water level of the tidal period
#' @param datumLow a numeric, lowest water level of the tidal period
#' @param tidalCycleLength a numeric, the length of time of the tidal cycle. This is set to 1 so that it estimates fractional flood time
#' 
#' @references Hickey, R. (2019). Tidal inundation modeling within GIS. Journal of Coastal Conservation, 23(3), 599-606.
#' @references Department of the Navy. Hydrographic Branch. 1994. Australian National Tide Tables: Australia, Papua & New Guinea, Australian Hydrographic Publication 11.
#'  
#' @return a numeric, flood time per tidal period
#' @export
def floodTimeTrig(z, datum_high, datum_low, tidal_period = 12.5):
  
  # If elevation is above the tidal range indation time is 0
  datum_high[z>=datum_high] = z[z>=datum_high]
  
  # If elevation is below inundation time is a full tidal cycle
  datum_low[z<=datum_low] = z[z<=datum_low]
  
  # Rising time over cell = 6.21 (A/pi - 1)	
  # where A = 2* pi - cos-1 [2 (height of cell – meanLowWater) / (meanHighWater – meanLowWater) - 1] radians
  A1 = 2 * np.pi - np.arccos(2 * (z - datum_low) / (datum_high - datum_low) - 1)
  rising_time = tidal_period / 2 * (A1 / np.pi - 1)
  
  # Falling time over cell = 6.21 (A/pi - 1) where
  # A = 2* - cos-1 [2 (height of cell – meanHighWater) / (meanLowWater – meanHighWater) - 1] radians
  A2 = 2 * np.pi - np.arccos(2 * (z - datum_high) / (datum_low - datum_high) - 1)
  falling_time = tidal_period / 2 * (A2 / np.pi - 1)
  
  # If between inundation time = abs (time rising - 6.21) + time falling
  inundation_time = np.abs(rising_time - tidal_period/2) + falling_time
  
  return inundation_time

Here we use the grid and tidal flow component settings to calculate the inundation time across the grid for one tidal period. 

In [ ]:
grid_inundation = floodTimeTrig(z, datum_high=tidal_range / 2 * np.ones_like(z), datum_low= -tidal_range / 2 * np.ones_like(z), tidal_period=tidal_period/3600)
plt.figure()
imshow_grid(
    grid, grid_inundation, cmap="YlGnBu", color_for_closed="g"
)

Using the deterministic relationship between biomass and annual inundation time, we can estimate the biomass and relate that back to roughness to demonstrate how vegetation roughness combines with bathymetry to influence tidal velocities.

In [ ]:
def solve_biomass_inundation_curve(
        inundation, 
        biomass_max=np.array([2500]), 
        inun_veg_max=np.array([3]), 
        inun_veg_min=np.array([-1]), 
        inun_veg_peak=np.nan
        ):
  # Stop the function if there are invalid parameters
    if (
        (np.any(biomass_max < 0)) 
        | (np.any(inun_veg_max <= np.fmax(inun_veg_min, inun_veg_peak))) 
        | (np.any(inun_veg_min >= np.fmax(inun_veg_max, inun_veg_peak)))
        ):
       return ("Invalid parameters were defined.")
    else:
        conditions = [
          np.isnan(inun_veg_peak),
          (z <= inun_veg_peak),
          (z > inun_veg_peak)
        ]
      # If there is no peak elevation for vegetation parabola is symmetric.
        midpoint = (inun_veg_max - inun_veg_min) / 2
        upper_z_veg_min = inun_veg_peak-((inun_veg_max-inun_veg_peak)) 
        lower_z_veg_max = inun_veg_peak+((inun_veg_peak-inun_veg_min))
      
        # From bmax, min, and max elevation limits, solve for parameters of a parabola.
        a_choices  = [
          (-inun_veg_min * biomass_max - inun_veg_max * biomass_max) / ((inun_veg_min - midpoint) * (-inun_veg_max + midpoint)),
          -((-inun_veg_min * biomass_max - lower_z_veg_max * biomass_max) / ((inun_veg_min - inun_veg_peak) * (-lower_z_veg_max + inun_veg_peak))),
          -((-upper_z_veg_min * biomass_max - inun_veg_max * biomass_max) / ((upper_z_veg_min - inun_veg_peak) * (-inun_veg_max + inun_veg_peak)))
        ]

        b_choices = [
          -(biomass_max / ((inun_veg_min - inun_veg_peak) * (-inun_veg_max + inun_veg_peak))),
          -(biomass_max / ((inun_veg_min - inun_veg_peak) * (-lower_z_veg_max + inun_veg_peak))),
          -(biomass_max / ((upper_z_veg_min - inun_veg_peak) * (-inun_veg_max + inun_veg_peak)))
        ]

        c_choices = [
          (inun_veg_min * inun_veg_max * biomass_max) / ((inun_veg_min - inun_veg_peak) * (inun_veg_max - inun_veg_peak)),
          (inun_veg_min * lower_z_veg_max * biomass_max) / ((inun_veg_min - inun_veg_peak) * (lower_z_veg_max - inun_veg_peak)),
          (upper_z_veg_min * inun_veg_max * biomass_max) / ((upper_z_veg_min - inun_veg_peak) * (inun_veg_max - inun_veg_peak))
        ]
        
        a = np.select(conditions, a_choices)
        b = np.select(conditions, b_choices)
        c = np.select(conditions, c_choices)
        # Apply parabolic function to elevation to calulate above ground biomass.
        abg = a * inundation + b * inundation**2 + c
        abg[abg < 0] = 0.0
    return abg

In [ ]:
max_inundation = np.ones_like(grid_inundation) * 11.5
min_inundation = np.zeros_like(grid_inundation)
inundation_at_peak_biomass = np.ones_like(grid_inundation) * 2.5
estimated_biomass[:] = solve_biomass_inundation_curve(grid_inundation, np.ones_like(grid_inundation) * 700, max_inundation, min_inundation, inundation_at_peak_biomass)
plt.figure()
imshow_grid(
    grid, estimated_biomass, cmap="YlGnBu", color_for_closed="g"
)


To calculate roughness, we typically use stem density and diameter to estimate Manning's coefficient based on laboratory and field studies. Since flow conditions on marshes are in the laminar to transitional range under normal tidal forcing, Manning's coefficient is not constant and varies with depth and velocity. However, for simplicity we will assume a constant drag coefficient that varies with biomass and depth using the following formula from Luhar and Nepf (2013) for emergent vegetation: 
$$ n = K_n (\frac{C_dah} {2g}\,) h^{1/6} $$

$n$ is the Mannin'g coefficient, $K_n$ is the units adjustment, $C_d$ is the bulk drag coefficient, $a$ is the frontal area per unit bed area calculated from the stem density and diameter, $g$ is the acceleration due to ravity, and $h$ is the water depth. We are neglecting vegetation flexibility since typical tidal velocities on the marsh plain are very small. 

In [ ]:
# Typical stem diameter for Sparina alterniflora in m
stem_diameter = 0.006
stem_height = 0.7
# density - biomass relationship from Morris and Haskin 1990
stem_density = np.array([0, 500, 2250])
biomass = np.array([0, 50, 700])
grid_stem_density = np.interp(estimated_biomass, biomass, stem_density)
grid_a = stem_diameter * grid_stem_density
drag_coefficient = 1.2

variable_mannings = (
    1 * (drag_coefficient * grid_a
    * stem_height / (2 * 9.81) * stem_diameter**(1/6))
)
variable_mannings[variable_mannings < 0.01] = 0.01
plt.figure()
imshow_grid(
    grid, variable_mannings, cmap="YlGnBu", color_for_closed="g"
)

In [ ]:
map_max_of_link_nodes_to_link(grid, variable_mannings, out=roughness)
# instantiate the TidalFlowCalculator
tfc_variable_roughness = TidalFlowCalculator(
    grid, tidal_range=tidal_range, tidal_period=tidal_period, roughness="roughness"
)

# run it
tfc_variable_roughness.run_one_step()

# make plots...
plot_tidal_flow(grid, resample=10)

The addition of variable roughness alters the flow paths across the marsh but not in a easily discernable way. Let's add some additional feedbacks to look at how marshes respond morphologically.

# Accretionary feedbacks

In real marshes, vegetation not only slows tidal velocities, it also traps sediments as well as builds up organic biomass in the soil. These allochotonous and autochotonous accretionary processes combine producing vegetation and tide mediated elevation changes over time.

In [ ]:
# Simple sediment deposition function (no hydraulics)

def estimate_allochtonous_deposition_per_year(estimated_biomass, peak_biomass=700, suspended_sediment=40, density_sediment=2650*(1-0.7)):
    suspended_sediment_kg_m3 = suspended_sediment / 1000
    depth = grid.at_node["mean_water__depth"]
    capture_efficiency = grid.at_node["biomass"] / peak_biomass
    #capture_efficiency[capture_efficiency<=0] = 0.05
    sed_deposition = suspended_sediment_kg_m3 * depth * capture_efficiency / density_sediment
    return sed_deposition

# Simple biologically generated accretion

def estimate_autochthonous_deposition_per_year(biomass, peak_biomass=700):
    max_auto_rate = 6 / 1000
    adjustment = max_auto_rate / peak_biomass
    auto_accretion_per_tide = biomass * adjustment
    return auto_accretion_per_tide

Now let's add in some (very) simplified accretion to look at how vegetation affects marsh morphology over several years.

In [ ]:
# Saving initial depth
intial_elev = grid.at_node["topographic__elevation"].copy()

In [ ]:
for _ in range(10):
    z += (estimate_allochtonous_deposition_per_year(estimated_biomass) + estimate_autochthonous_deposition_per_year(estimated_biomass))
    grid_inundation = floodTimeTrig(z, datum_high=tidal_range / 2 * np.ones_like(z), datum_low= -tidal_range / 2 * np.ones_like(z), tidal_period=tidal_period/3600)
    estimated_biomass = solve_biomass_inundation_curve(grid_inundation, np.ones_like(grid_inundation) * 700, max_inundation, min_inundation, inundation_at_peak_biomass)
    grid_stem_density = np.interp(estimated_biomass, biomass, stem_density)
    grid_a = stem_diameter * grid_stem_density
    variable_mannings = (
        1 * (drag_coefficient * grid_a
        * stem_height / (2 * 9.81) * stem_diameter**(1/6))
    )
    variable_mannings[variable_mannings < 0.01] = 0.01
    map_max_of_link_nodes_to_link(grid, variable_mannings, out=roughness)
    tfc_variable_roughness.run_one_step()    

In [ ]:
plot_tidal_flow(grid, resample=10)

In [ ]:
dz_final = grid.at_node["topographic__elevation"] - intial_elev
imshow_grid(grid, dz_final)

We know this is a dramatically simplified approach but it illustrates how many marsh models consider vegetation effects. In the next section we will explore a more complex consideration of marsh processes. 

# Python MarshMorpho2d
The USACE Ecomod team has worked to develop a preliminary Python implemntation of the Mariotti's (2018) MarshMorpho2D and integrate this with LandLab. This is a first step towards implementing a componentized implementation a coastal wetland ecogeomorphic model in Landlab. The goal is to facilitate advancement of coastal wetland modeling as well as enable these models to be coupled with existing data and modeling outputs.

Additional parmeters are needed in addition to the LandLab tidal_flow_calculator, mainly the definition of closed vs open boudaries to define the erodable domain.


In [ ]:
from pyMM2D.pymarshmorpho2d import pymm2d
from tqdm import tqdm
import warnings # This is to help get around some versioning issues.
warnings.simplefilter(action='ignore', category=FutureWarning)


grid.at_node["topographic__elevation"] = np.copy(z) # setup z 

# grid boundaries. For this example we are closing all boundaries except the "bottom" boundary to allow
# tidal exchange.
grid.set_closed_boundaries_at_grid_edges(right_is_closed=True,
                                               top_is_closed=True,
                                               left_is_closed=True,
                                               bottom_is_closed=False)

# function to define the closed boundaries and also stop the very edge of the boundary from eroding. 
def complete_domain(model_domain):
    checkDomain = np.reshape(model_domain, grid.shape)
    domainIndex = np.reshape(range(0,len(model_domain)), grid.shape)
    if len(checkDomain[checkDomain[:,0] == 2]) == len(checkDomain[:,0])-2:
        print("The model is open along the left edge")
        checkDomain[:,0] = 2
    if len(checkDomain[checkDomain[:,-1] == 2]) == len(checkDomain[:,-1])-2:
        print("The model is open along the Right edge")
        checkDomain[:,-1] = 2
    updated_bndry_nodes = domainIndex[checkDomain ==2]
    return(checkDomain, updated_bndry_nodes)


model_domain = np.copy(z)
model_domain[:] = 1
model_domain, bndryNodes = complete_domain(model_domain)

# define the elevation at the active domain boundaries. If this is not defined, excessive erosion will occure.
boundaryTopo = z.flatten()[bndryNodes]

Once the model domains and erodable area has been defined we can call the MarshMorpho2D package (pymm2d) and provide the necessary information to run the model.

In [ ]:
mev = pymm2d(grid, model_domain=model_domain, boundaryValues = boundaryTopo, runMEM=False, roughness = tfc_variable_roughness._roughness)

print("Starting main loop")
for i in tqdm(range(10), colour = "green"):
# for i in range(10):
    slr = 4 # annual sea level rise in mm
    mev.run_one_step(timeStep = 1, round = i, model_domain = model_domain, relative_sea_level_rise_rate_mmPerYr = slr,
                      roughness = tfc_variable_roughness._roughness) 
    grid.at_node["topographic__elevation"] = mev._elev
    if i == 0:
        testEle = (mev._elev) # record the inital state for comparison

# make plots...
plot_tidal_flow(grid, resample=10)